# **Capitulo 2 – Training Models**

# Setup

Como hicimos en Notebook anteriores, vamos a definir los tamaños de fuente por defecto para que las figuras queden más bonitas:

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

Y vamos a crear la carpeta `images/training_linear_models` (si no existe ya), y definir la función `save_fig()` que se utiliza a través de este cuaderno para guardar las figuras en alta resolución para el libro:

In [ ]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "training_linear_models"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Hasta ahora hemos tratado los modelos de aprendizaje automático y sus algoritmos de entrenamiento como cajas negras. Si ha realizado algunos de los ejercicios de los capítulos anteriores, puede que se haya sorprendido de lo mucho que puede hacer sin saber nada de lo que hay bajo el capó: ha optimizado un sistema de regresión, ha mejorado un clasificador de imágenes de dígitos e incluso ha construido un clasificador de spam desde cero, todo ello sin saber cómo funcionan realmente. De hecho, en muchas situaciones no es necesario conocer los detalles de la implementación.

Sin embargo, comprender bien cómo funcionan las cosas puede ayudarle a encontrar rápidamente el modelo adecuado, el algoritmo de entrenamiento correcto y un buen conjunto de hiperparámetros para su tarea. Entender qué hay bajo el capó también le ayudará a depurar problemas y a realizar análisis de errores de forma más eficiente. Por último, la mayoría de los temas discutidos en este capítulo serán esenciales para entender, construir y entrenar redes neuronales (discutido en la Parte II de este libro). En este capítulo comenzaremos analizando el modelo de regresión lineal, uno de los modelos más simples que existen. Discutiremos dos formas muy diferentes de entrenarlo:

- Utilizando una ecuación de "forma cerrada "1 que calcula directamente los parámetros del modelo que mejor se ajustan al conjunto de entrenamiento (es decir, los parámetros del modelo que minimizan la función de coste sobre el conjunto de entrenamiento).

- Utilizando un método de optimización iterativo llamado descenso de gradiente (DG) que ajusta gradualmente los parámetros del modelo para minimizar la función de coste sobre el conjunto de entrenamiento, convergiendo finalmente al mismo conjunto de parámetros que el primer método.

A continuación estudiaremos la regresión polinómica, un modelo más complejo que puede ajustarse a conjuntos de datos no lineales. Como este modelo tiene más parámetros que la regresión lineal, es más propenso a sobreajustar los datos de entrenamiento. Exploraremos cómo detectar si éste es o no el caso utilizando curvas de aprendizaje y, a continuación, estudiaremos varias técnicas de regularización que pueden reducir el riesgo de sobreajuste del conjunto de entrenamiento.

Por último, examinaremos otros dos modelos que se utilizan habitualmente para tareas de clasificación: la regresión logística y la regresión softmax.

En este capítulo habrá bastantes ecuaciones matemáticas, utilizando nociones básicas de álgebra lineal y cálculo. Para entender estas ecuaciones, necesitarás saber qué son los vectores y las matrices; cómo transponerlos, multiplicarlos e invertirlos; y qué son las derivadas parciales. Si no estás familiarizado con estos conceptos, consulta los tutoriales introductorios de álgebra lineal y cálculo disponibles como cuadernos Jupyter en el material complementario en línea. Para aquellos que son realmente alérgicos a las matemáticas, todavía debe ir a través de este capítulo y simplemente omitir las ecuaciones, con suerte, el texto será suficiente para ayudarle a entender la mayoría de los conceptos.

# Regresión lineal

Esto puede expresarse de manera más concisa utilizando una forma vectorizada, como se muestra en la ecuación.

$$\hat{y} = ℎ_{θ}(x) = θ · x$$

En esta ecuación:

- $ℎ_{θ}$ es la función de hipótesis, utilizando los parámetros del modelo $θ$.
- $θ$ es el vector de parámetros del modelo, que contiene el término de sesgo $θ₀$ y los pesos de las características $θ₁$ a $θn$.
- $x$ es el vector de características de la instancia, que contiene $x₀$ a $x_n$, con $x₀$ siempre igual a 1.
- $θ · x$ es el producto punto de los vectores $θ$ y $x$, que es igual a $θ₀x₀ + θ₁x₁ + θ₂x₂ + ... + θnxn$


En el Machine Learning, los vectores a menudo se representan como vectores columna, que son arreglos bidimensionales con una sola columna. Si $θ$ y $x$ son vectores columna, entonces la predicción es $\hat{y} = θ^⊺ \times x$, donde $θ^⊺$ es la transpuesta de $θ$ (un vector fila en lugar de un vector columna) y $θ^⊺ \times x$ es la multiplicación de matrices de $θ^⊺$ y $x$. Por supuesto, es la misma predicción, excepto que ahora se representa como una matriz de una sola celda en lugar de un valor escalar. En este libro, utilizaré esta notación para evitar cambiar entre productos punto y multiplicaciones de matrices.

Bien, eso es el modelo de regresión lineal, ¿pero cómo lo entrenamos? Recordemos que entrenar un modelo significa establecer sus parámetros de modo que el modelo se ajuste mejor al conjunto de entrenamiento. Para este propósito, primero necesitamos una medida de qué tan bien (o mal) el modelo se ajusta a los datos de entrenamiento. Por lo tanto, para entrenar un modelo de regresión lineal, necesitamos encontrar el valor de $θ$ que minimiza el RMSE. En la práctica, es más sencillo minimizar el error cuadrático medio (MSE, por sus siglas en inglés) que el RMSE, y conduce al mismo resultado (ya que el valor que minimiza una función positiva también minimiza su raíz cuadrada).

Los algoritmos de machine learning a menudo optimizan una función de pérdida diferente durante el entrenamiento que la medida de rendimiento utilizada para evaluar el modelo final. Esto generalmente se debe a que la función es más fácil de optimizar y/o porque tiene términos adicionales necesarios solo durante el entrenamiento (por ejemplo, para la regularización). Una buena métrica de rendimiento es lo más cercana posible al objetivo final del negocio. Una buena pérdida de entrenamiento es fácil de optimizar y está fuertemente correlacionada con la métrica. Por ejemplo, los clasificadores a menudo se entrenan utilizando una función de costo como el log loss (como verás más adelante en este capítulo), pero se evalúan utilizando precisión/recall. El log loss es fácil de minimizar, y hacerlo generalmente mejorará la precisión/recall.

El MSE de una hipótesis de regresión lineal $h_{θ}$ en un conjunto de entrenamiento X se calcula utilizando la Ecuación 4-3.

Ecuación 4-3. Función de costo MSE para un modelo de regresión lineal
$$MSE(X, h_{θ}) = \frac{1}{m} \sum_{i=1}^{m} (θ^⊺x_{i} - y_{i})^{2}$$

La única diferencia es que escribimos $h_{θ}$ en lugar de solo $h$ para dejar claro que el modelo está parametrizado por el vector $θ$. Para simplificar las notaciones, simplemente escribiremos $MSE(θ)$ en lugar de $MSE(X, h_{θ})$.

## La Ecuación Normal

Para encontrar el valor de $θ$ que minimiza el $MSE$, existe una solución de forma cerrada, es decir, una ecuación matemática que da el resultado directamente. Esto se conoce como la ecuación normal.

$$\hat{θ} = (X^⊺X)^{−1}X^⊺y$$

En esta ecuación:

$\hat{θ}$ es el valor de $θ$ que minimiza la función de costo.
$y$ es el vector de valores objetivo que contiene $y^{(1)}$ a $y^{(m)}$

In [ ]:
import numpy as np

np.random.seed(42)
m = 100 # número de instancias
X = 2 * np.random.rand(m, 1)  
y = 4 + 3 * X + np.random.randn(m, 1) 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(X, y, "b.")
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.axis([0, 2, 0, 15])
plt.grid()
save_fig("generated_data_plot")
plt.show()

Ahora vamos a calcular $θ$ utilizando la ecuación Normal. Utilizaremos la función inv() del módulo álgebra lineal de NumPy (np.linalg) para calcular la inversa de una matriz, y el método dot() para la multiplicación de matrices:

In [ ]:
from sklearn.preprocessing import add_dummy_feature

X_b = add_dummy_feature(X)  # añade x0 = 1 a cada instancia
theta_best = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y

El operador @ realiza la multiplicación de matrices. Si A y B son matrices NumPy, entonces A @ B es equivalente a np.matmul(A, B). Muchas otras bibliotecas, como TensorFlow, PyTorch y JAX, también soportan el operador @. Sin embargo, no se puede utilizar @ en arrays Python puros (es decir, listas de listas).

La función que hemos utilizado para generar los datos es $y = 4 + 3x1 + ruido \space gaussiano$. Veamos lo que encontró la ecuación:

In [ ]:
theta_best

Habríamos esperado $θ_0 = 4$ y $θ_1 = 3$ en lugar de $θ_0 = 4,215$ y $θ_1 = 2,770$. Se acercaba bastante, pero el ruido hacía imposible recuperar los parámetros exactos de la función original. Cuanto más pequeño y ruidoso es el conjunto de datos, más difícil resulta.
Ahora podemos hacer predicciones utilizando $θ$:

In [ ]:
X_new = np.array([[0], [2]])
X_new_b = add_dummy_feature(X_new)  # añade x0 = 1 a cada instancia
y_predict = X_new_b @ theta_best
y_predict

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))  
plt.plot(X_new, y_predict, "r-", label="Predictions")
plt.plot(X, y, "b.")


plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.axis([0, 2, 0, 15])
plt.grid()
plt.legend(loc="upper left")
save_fig("linear_model_predictions_plot")

plt.show()

Realizar una regresión lineal con Scikit-Learn es relativamente sencillo:

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

In [ ]:
lin_reg.predict(X_new)

La clase `LinearRegression` se basa en la función `scipy.linalg.lstsq()` (el nombre significa "mínimos cuadrados"), que podrías llamar directamente:

In [ ]:
theta_best_svd, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond=1e-6)
theta_best_svd

Esta función calcula $\mathbf{X}^+\mathbf{y}$, donde $\mathbf{X}^{+}$ es la _pseudoinversa_ de $\mathbf{X}$ (concretamente la inversa de Moore-Penrose). Puede utilizar `np.linalg.pinv()` para calcular la pseudoinversa directamente:

In [ ]:
np.linalg.pinv(X_b) @ y

La pseudoinversa se calcula utilizando una técnica estándar de factorización de matrices llamada descomposición en valores singulares (SVD) que puede descomponer la matriz del conjunto de entrenamiento $X$ en la multiplicación de tres matrices $U Σ V^⊺$ (ver numpy.linalg.svd()). La pseudoinversa se calcula como $X^+ = VΣ^+U^⊺$. Para calcular la matriz $Σ^+$, el algoritmo toma Σ y establece a cero todos los valores menores que un valor umbral diminuto, luego reemplaza todos los valores no nulos con su inverso y finalmente transpone la matriz resultante. Este enfoque es más eficiente que calcular la Ecuación Normal, además de que maneja bien los casos límite: de hecho, la Ecuación Normal puede no funcionar si la matriz $X^⊺X$ no es invertible (es decir, singular), como si $m < n$ o si algunas características son redundantes, pero la pseudoinversa siempre está definida.

Complejidad Computacional
La Ecuación Normal calcula la inversa de X^⊺X, que es una matriz de $(n + 1) × (n + 1)$ (donde n es el número de características). La complejidad computacional de invertir una matriz de este tipo es típicamente de alrededor de $O(n^2.4)$ a $O(n^3)$, dependiendo de la implementación. En otras palabras, si duplicas el número de características, multiplicarás el tiempo de cómputo aproximadamente por $2^2.4 = 5.3 a 2^3 = 8$.


El enfoque SVD utilizado por la clase LinearRegression de Scikit-Learn es aproximadamente $O(n^2)$. Si se duplica el número de características, el tiempo de cálculo se multiplica aproximadamente por 4.

Tanto la ecuación Normal como el enfoque SVD se vuelven muy lentos cuando el número de características aumenta (por ejemplo, 100.000). Por el lado positivo, ambos son lineales con respecto al número de instancias del conjunto de entrenamiento (son $O(m)$), por lo que manejan conjuntos de entrenamiento grandes de forma eficaz, siempre que quepan en la memoria.

Además, una vez entrenado el modelo de regresión lineal (mediante la ecuación Normal o cualquier otro algoritmo), las predicciones son muy rápidas: la complejidad computacional es lineal con respecto tanto al número de instancias sobre las que se quieren hacer predicciones como al número de características. En otras palabras, hacer predicciones sobre el doble de casos (o el doble de características) llevará aproximadamente el doble de tiempo.

Ahora veremos una forma muy distinta de entrenar un modelo de regresión lineal, que es más adecuada para los casos en los que hay un gran número de características o demasiadas instancias de entrenamiento para que quepan en la memoria